In [1]:
import requests
import dspy
import os
from dotenv import load_dotenv
import json
from openai import OpenAI

c:\Users\Home\anaconda3\envs\spcopenai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
gpt4 = dspy.OpenAI(model='gpt-4', api_key=OPENAI_API_KEY)
dspy.configure(lm=gpt4)
client = OpenAI()

In [43]:
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": "perplexity/sonar-small-chat", # Optional
    "messages": [
      {"role": "user", "content": "List 5 U.S stocks (and their tickers) trending today. They must be valid stocks."}
    ]
  })
)

In [44]:
response.json()

{'choices': [{'message': {'role': 'assistant',
    'content': "1. Amazon (AMZN) - The largest online retailer is currently trending, with its knee-jerk response to counterfeit merchandise impacting sales for various third-party sellers on the platform, resulting in a temporary slowdown in earnings.\n\n2. Tesla, Inc. (TSLA) - Tesla is currently relevant due to regulatory board meetings occurring during the first week of Tesla's Q4 earnings report. The conversion from a Kal-El to True Quantum computer may delay production, and the price of Tesla stock fell after reports of Elon Musk's divestment from his stake in Bitcoin and his plan to invest in Dogecoin.\n\n3. GameStop (GME) - GameStop, a video game retailer, has recently been trending due to its involvement in a short squeeze caused by the subreddit r/WallStreetBets. The stock price soared in the past week, and the company is now exploring the possibility of spinning off its bookstore chain, GameStop Education.\n\n4. NIO (NIO) - NIO, 

In [45]:
text = response.json()["choices"][0]["message"]["content"]
text

"1. Amazon (AMZN) - The largest online retailer is currently trending, with its knee-jerk response to counterfeit merchandise impacting sales for various third-party sellers on the platform, resulting in a temporary slowdown in earnings.\n\n2. Tesla, Inc. (TSLA) - Tesla is currently relevant due to regulatory board meetings occurring during the first week of Tesla's Q4 earnings report. The conversion from a Kal-El to True Quantum computer may delay production, and the price of Tesla stock fell after reports of Elon Musk's divestment from his stake in Bitcoin and his plan to invest in Dogecoin.\n\n3. GameStop (GME) - GameStop, a video game retailer, has recently been trending due to its involvement in a short squeeze caused by the subreddit r/WallStreetBets. The stock price soared in the past week, and the company is now exploring the possibility of spinning off its bookstore chain, GameStop Education.\n\n4. NIO (NIO) - NIO, a leading electric vehicle company, is trending after news bro

In [49]:
class DeriveStocks(dspy.Signature):
    """Given a message, derive the stocks and the tickers mentioned in the message.
    If the ticker is not mentioned, the model will try to predict the ticker based on the stock name."""

    message = dspy.InputField()
    stock1 = dspy.OutputField(desc="The first company name")
    ticker1 = dspy.OutputField(desc="The first company ticker")
    stock2 = dspy.OutputField(desc="The second company name")
    ticker2 = dspy.OutputField(desc="The second company ticker")
    stock3 = dspy.OutputField(desc="The third company name")
    ticker3 = dspy.OutputField(desc="The third company ticker")
    stock4 = dspy.OutputField(desc="The fourth company name")
    ticker4 = dspy.OutputField(desc="The fourth company ticker")
    stock5 = dspy.OutputField(desc="The fifth company name")
    ticker5 = dspy.OutputField(desc="The fifth company ticker")

In [50]:
stock_derive = dspy.Predict(DeriveStocks)
answer = stock_derive(message=text)
answer

Prediction(
    stock1='Amazon',
    ticker1='AMZN',
    stock2='Tesla, Inc.',
    ticker2='TSLA',
    stock3='GameStop',
    ticker3='GME',
    stock4='NIO',
    ticker4='NIO',
    stock5='Airbnb',
    ticker5='ABNB'
)

In [53]:
answer.toDict()

{'stock1': 'Amazon',
 'ticker1': 'AMZN',
 'stock2': 'Tesla, Inc.',
 'ticker2': 'TSLA',
 'stock3': 'GameStop',
 'ticker3': 'GME',
 'stock4': 'NIO',
 'ticker4': 'NIO',
 'stock5': 'Airbnb',
 'ticker5': 'ABNB'}

In [11]:
class DeriveTicker(dspy.Signature):
    """Given a search query containing a company name or ticker, derive the proper stock market ticker for the intended company."""

    search_query = dspy.InputField()
    ticker = dspy.OutputField(desc="The company ticker in one word, or N/A if the company is not found")

In [14]:
stock_derive = dspy.Predict(DeriveTicker)
answer = stock_derive(search_query="A not real company")
answer

Prediction(
    ticker='N/A'
)